# Optibook Manual

This notebook contains examples for **all interactions** you can do with optibook (that means inserting orders, getting your positions, etc.). You should use this notebook as a reference/documenation for the system later on when you write more extensive algorithms.

First we do some setup and import the optibook client, which is used to connect to the optibook exchange. If everything is setup correctly, you should see the line "Setup was successful." being printed.

In [22]:
from optibook.synchronous_client import Exchange

import logging
logger = logging.getLogger('client')
logger.setLevel('ERROR')

print("Setup was successful.")

Setup was successful.


### Define Your Instrument

An instrument is the term we use for the 'thing' that we are trading. As such, if we are trading BMW stocks, we would call those an instrument. However, BMW options would be a different instrument. 

This field determines which instrument we are trading. By changing it, you will insert trades for a different instrument. You can see all available instruments by looking at the dropdown menu labeled "Instruments" on the visualizer.

In [18]:
instrument_id = 'PHILIPS_A'

### Connect to Exchange

In [23]:
e = Exchange()
a = e.connect()

# you can also define host/user/pass yourself
# when not defined, it is taken from ~/.optibook file if it exists
# if that file does not exists, an error is thrown

#e = Exchange(host='host-to-connect-to')
#a = e.connect(username='your-username', password='your-password')


2022-04-03 09:14:50,754 [asyncio   ] [MainThread  ] Using selector: EpollSelector


## Outstanding Orders, Trades, Current Positions and PnL

In [10]:
# Returns all currently outstanding orders
orders = e.get_outstanding_orders(instrument_id)
for o in orders.values():
    print(o)

In [5]:
# Returns all trades you have done since the last time this function was called
trades = e.poll_new_trades(instrument_id)
for t in trades:
    print(f"[TRADED {t.instrument_id}] price({t.price}), volume({t.volume}), side({t.side})")

In [6]:
# Returns all trades you have done since since the instantiation of the Exchange
trades = e.get_trade_history(instrument_id)
for t in trades:
    print(f"[TRADED {t.instrument_id}] price({t.price}), volume({t.volume}), side({t.side})")

In [11]:
# Returns all current positions
positions = e.get_positions()
for p in positions:
    print(p, positions[p])

PHILIPS_A 12
PHILIPS_B 12


2022-04-03 08:42:00,401 [client    ] [Thread-5    ] Forcing a disconnect due to an error: Closing connection because someone else logged in with the same credentials. Only one session may be active at the same time.


In [8]:
# Returns all current positions with cash invested
positions = e.get_positions_and_cash()
for p in positions:
    print(p, positions[p])

PHILIPS_A {'volume': 36, 'cash': -3035.8119999999994}
PHILIPS_B {'volume': 0, 'cash': 0.0}


In [9]:
# Returns Current PnL based on last Traded Price
pnl = e.get_pnl()
print(pnl)

-638.2119999999991


## Order Book and Public Trade Ticks

In [10]:
book = e.get_last_price_book(instrument_id)
print(book.timestamp)

2022-04-03 03:42:11.765972


In [11]:
# Returns all public tradeticks since the last time this function was called
tradeticks = e.poll_new_trade_ticks(instrument_id)
for t in tradeticks:
    print(f"[{t.instrument_id}] price({t.price}), volume({t.volume}), aggressor_side({t.aggressor_side}), buyer({t.buyer}), seller({t.seller})")

In [12]:
# Returns all public tradeticks since the instantiation of the Exchange
tradeticks = e.get_trade_tick_history(instrument_id)
for t in tradeticks:
    print(t)

In [13]:
# See all your outstanding orders
outstanding = e.get_outstanding_orders(instrument_id)
for o in outstanding.values():
    print(f"Outstanding order: order_id({o.order_id}), instrument_id({o.instrument_id}), price({o.price}), volume({o.volume}), side({o.side})")

In [16]:
from sortedcontainers import SortedDict
import time

class LOB:
    def __init__(self):
        self.totalPositionsPA = 0
        self.totalPositionsPB = 0
        self.orderTracker = dict()
        self.actionCount = 0
        self.timeEpoch = time.time()
        
    def addOrder(self, e, strategy: int, instrument_id: str, price: float, volume: int, side: str, order_type: str) -> int:
        self.timeHeat()
        order_id = e.insert_order(instrument_id, price=price, volume=volume, side=side, order_type=order_type)
        self.orderTracker[order_id] = (strategy, instrument_id, price, volume, side, order_type)
        return order_id
        
    def deleteOrder(self, e, instrument_id: str, order_id: int) -> bool:
        self.timeHeat()
        success = e.delete_order(instrument_id, order_id=order_id)
        if success:
            del self.orderTracker[order_id]
            return -1
        else:
            disorder = self.orderTracker[order_id]
            if disorder == 'ask':
                return self.addOrder(e, disorder[0], disorder[1], disorder[2], disorder[3], 'bid', disorder[5])
            else:
                return self.addOrder(e, disorder[0], disorder[1], disorder[2], disorder[3], 'ask', disorder[5])
            
    
    def hedgeAlarm(self):
        sumPos = self.totalPositionsPA + self.totalPositionsPB
        if abs(sumPos) > 40:
            return True
        return False
        
    def timeHeat(self) -> bool:
        if self.actionCount <= 25:
            self.actionCount +=1
            return True
        else:
            if time.time() - self.timeEpoch >= 1:
                self.timeEpoch = time.time()
                self.actionCount = 0
                return True
            else:
                time.sleep(max(1 - (time.time() - self.timeEpoch), 0))
    


In [17]:
orders = LOB()

## Inserting and Deleting Orders

In [18]:
# Insert bid limit order - This trades against any current orders, and any remainders become new resting orders in the book
# Use this to buy.
result = orders.addOrder(e,1, instrument_id, price=300, volume=5, side='bid', order_type='limit')
print(f"Order Id: {result}")

Order Id: 355130


2022-04-03 03:42:59,790 [client    ] [Thread-4    ] Notification [limit-checker]: You breached the risk limits. Your positions have been reduced automatically to bring them within limits.
Automatically reducing your positions comes at a small cost. Your positions are reduced to 90% of the limit at a premium of 0% compared to the fair price of the instrument.
The following limits were breached:
[per_underlying_hedged]
instrument=PHILIPS_A limit_name=delta limit_value=40, your_value=41.0


In [21]:
# Insert ask limit order - This trades against any current orders, and any remainders become new resting orders in the book
# Use this to sell.
result = orders.addOrder(e, 1, instrument_id, price=3000, volume=1, side='ask', order_type='limit')
print(f"Order Id: {result}")

Order Id: 337435


In [16]:
# Insert bid IOC - This can trade against any resting volume but does not remain in the book
# Use this to buy.
result = e.insert_order(instrument_id, price=445.0, volume=1, side='bid', order_type='ioc')
print(f"Order Id: {result}")

Order Id: 534174


In [17]:
# Insert ask IOC - This can trade against any resting volume but does not remain in the book
# Use this to sell.
result = e.insert_order(instrument_id, price=430.0, volume=1, side='ask', order_type='ioc')
print(f"Order Id: {result}")

Order Id: 534175


In [29]:
# Attempt to delete inserted order by order_id
order_id = 387062
result = orders.deleteOrder(e, instrument_id, order_id=order_id)
print()
print(f"Deleted order id {order_id}: {result}")


Deleted order id 387062: -1


In [19]:
# Change volume for existing order
order_id = 5
new_volume = 16
result = e.amend_order(instrument_id, order_id=order_id, volume=new_volume)
print(f"Changed volume for order id {order_id} to {new_volume} lots: {result}")

Changed volume for order id 5 to 16 lots: False


In [27]:
# Delete all outstanding orders
outstanding = e.get_outstanding_orders(instrument_id)
for o in outstanding.values():
    result = e.delete_order(instrument_id, order_id=o.order_id)
    print(f"Deleted order id {o.order_id}: {result}")

## 'Hack' Out of Positions

In [25]:
# Get out of all positions you are currently holding, regarless of the loss involved. That means selling whatever
# you are long, and buying-back whatever you are short. Be sure you know what you are doing when you use this logic.
print(e.get_positions())
for s, p in e.get_positions().items():
    if p > 0:
        e.insert_order(s, price=1, volume=p, side='ask', order_type='ioc')
    elif p < 0:
        e.insert_order(s, price=100000, volume=-p, side='bid', order_type='ioc')  
print(e.get_positions())

{'PHILIPS_A': 0, 'PHILIPS_B': 0}
{'PHILIPS_A': 0, 'PHILIPS_B': 0}


2022-04-03 09:15:05,607 [client    ] [Thread-8    ] Forcing a disconnect due to an error: Closing connection because someone else logged in with the same credentials. Only one session may be active at the same time.
